# Amplitude behavior investigation

Intensity function for two-pseudoscalar system:

```{math}
:class: full-width
I(\Omega,\Phi) = 2\kappa\sum_{k}\left(
  (1-P_{\gamma})\left|\sum_{l,m}[l]_{m;k}^{(-)}\Re[Z_{l}^{m}(\Omega,\Phi)]\right|^2
  +(1-P_{\gamma})\left|\sum_{l,m}[l]_{m;k}^{(+)}\Im[Z_{l}^{m}(\Omega,\Phi)]\right|^2
  +(1+P_{\gamma})\left|\sum_{l,m}[l]_{m;k}^{(+)}\Re[Z_{l}^{m}(\Omega,\Phi)]\right|^2
  +(1+P_{\gamma})\left|\sum_{l,m}[l]_{m;k}^{(-)}\Im[Z_{l}^{m}(\Omega,\Phi)]\right|^2
\right)
```

where $Z_{l}^{m}(\Omega,\Phi)=Y_{l}^{m}(\Omega)e^{-i\Phi}$ is a phase-rotated spherical harmonic, $\Omega$ is the solid angle, $\Phi$ is the angle between the production and polarization planes,  $P_{\gamma}$ is the polarization magnitude, $[l]$ are the partial wave amplitudes, $m$ is the associated m-projection, $k$ refers to a spin flip ($k=1$) or non-flip ($k=0$) at the nucleon vertex, and $\kappa$ is an overall phase space factor.

## Main intensity definition

In [ ]:
from __future__ import annotations

import sympy as sp
from ampform.io import aslatex
from ampform.sympy import PoolSum
from IPython.display import Math
from sympy.functions.special.spherical_harmonics import Ynm

In [ ]:
from sympy.printing.precedence import PRECEDENCE
from sympy.printing.printer import Printer


def _print_Indexed_latex(self, printer, *args):
    base = printer._print(self.base)
    indices = ", ".join(map(printer._print, self.indices))
    return f"{base}_{{{indices}}}"


sp.Indexed.precedence = PRECEDENCE["Pow"] - 1
sp.Indexed._latex = _print_Indexed_latex
Printer._global_settings["gothic_re_im"] = True

In [ ]:
k, l, m = sp.symbols("k l m")
phi, theta, Phi = sp.symbols("phi theta Phi", real=True)
kappa = sp.Symbol("kappa")
Py = sp.Symbol("P_gamma")

In [ ]:
from ampform.sympy import (
    UnevaluatedExpression,
    create_expression,
    implement_doit_method,
)


@implement_doit_method
class Znm(UnevaluatedExpression):
    is_commutative = True
    is_real = False

    def __new__(cls, n, m, theta, phi, Phi, **hints) -> Znm:
        return create_expression(cls, n, m, theta, phi, Phi, **hints)

    def evaluate(self) -> sp.Mul:
        n, m, theta, phi, Phi = self.args
        return Ynm(n, m, theta, phi) * sp.exp(-sp.I * Phi)

    def _latex(self, printer, *args):
        n, m, theta, phi, Phi = map(printer._print, self.args)
        return Rf"Z_{n}^{m}({theta}, {phi}, {Phi})"

In [ ]:
znm = Znm(l, m, theta, phi, Phi)
Math(aslatex({znm: znm.doit()}))

In [ ]:
assert sp.im(znm) != 0

In [ ]:
from qrules.quantum_numbers import arange


def create_range(__min, __max) -> tuple[sp.Rational, ...]:
    return tuple(sp.Rational(x) for x in arange(__min, __max + 1))


max_l = 2
k_values = [0]
l_range = [max_l]  # create_range(0, max_l)
m_range = create_range(-max_l, max_l)
lp = sp.IndexedBase("[l]^{(+)}", complex=True)
lm = sp.IndexedBase("[l]^{(-)}", complex=True)

intensity_expr = (
    2
    * kappa
    * PoolSum(
        (1 - Py)
        * sp.Abs(PoolSum(lm[m, k] * sp.re(znm), (l, l_range), (m, m_range))) ** 2
        + (1 - Py)
        * sp.Abs(PoolSum(lp[m, k] * sp.im(znm), (l, l_range), (m, m_range))) ** 2
        + (1 + Py)
        * sp.Abs(PoolSum(lp[m, k] * sp.re(znm), (l, l_range), (m, m_range))) ** 2
        + (1 + Py)
        * sp.Abs(PoolSum(lm[m, k] * sp.im(znm), (l, l_range), (m, m_range))) ** 2,
        (k, k_values),
    )
)
intensity_expr

In [ ]:
sp.expand_func(intensity_expr.doit())

## Suggested parameter values

In [ ]:
lp_defaults = {lp[m, k]: 0 for k in k_values for m in m_range}
lm_defaults = {lm[m, k]: 1 for k in k_values for m in m_range}
parameter_defaults = {
    Py: 1,
    kappa: sp.pi,
    **lp_defaults,
    **lm_defaults,
}

In [ ]:
substituted_expr = intensity_expr.doit().xreplace(parameter_defaults)
substituted_expr

In [ ]:
sp.expand_func(substituted_expr).simplify()

## Phase space investigation

In [ ]:
free_symbols = sorted(intensity_expr.doit().free_symbols, key=str)
amp_symbols = [s for s in free_symbols if isinstance(s, sp.Indexed)]
arguments = (theta, phi, Phi, Py, *amp_symbols)
plotted_expr = intensity_expr.doit().xreplace({kappa: sp.pi}).expand(func=True)
intensity_func = sp.lambdify(arguments, plotted_expr, "numpy", cse=True)
intensity_func

In [ ]:
import numpy as np

cos_theta_array, phi_array = np.meshgrid(
    np.linspace(-1, +1, num=200),
    np.linspace(0, 2 * np.pi, num=400),
)
theta_array = np.arccos(cos_theta_array)

In [ ]:
import re
from collections import defaultdict

import ipywidgets as w

# Create sliders for each symbol/argument
sliders = dict(
    Phi_val=w.FloatSlider(
        description="Φ",
        min=0,
        max=np.pi,
        value=1,
        step=np.pi / 20,
    ),
    Py_val=w.FloatSlider(
        description="Pγ",
        min=0,
        max=1,
        value=1,
    ),
)
for s in free_symbols:
    if not isinstance(s, sp.Indexed):
        continue
    if not s.name.startswith("[l]"):
        continue
    key = s.name
    sliders[f"{key}_mag"] = w.FloatSlider(
        description="magnitude",
        min=0,
        max=2,
        value=1 if "+" in s.name else 0,
    )
    sliders[f"{key}_phi"] = w.FloatSlider(
        description="phase",
        min=0,
        max=2,
    )

# Group sliders so they can be organised in a layout for the UI
l_sliders = defaultdict(list)
for key in sliders:
    if not key.startswith("[l]^{("):
        continue
    if not key.endswith("_mag"):
        continue
    l_str = key.strip("_mag")
    sign = l_str[6]
    indices = eval(l_str[9:])
    superscript = f"<sup>({sign})</sup>"
    subscript = f"<sub>{','.join(f'{i:+d}' if i else str(i) for i in indices)}</sub>"
    label = f"[𝑙]{superscript}{subscript}"
    row = w.HBox([w.HTML(label), sliders[f"{l_str}_mag"], sliders[f"{l_str}_phi"]])
    l_sliders[sign].append(row)

# Create the UI
vbox_items = [
    sliders["Phi_val"],
    sliders["Py_val"],
    w.Accordion(
        [w.VBox(l_sliders["+"]), w.VBox(l_sliders["-"])],
        selected_index=0,
        titles=["Positive [𝑙]", "Negative [𝑙]"],
    ),
]
UI = w.VBox(vbox_items)

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(7, 4))
ax.set_xlabel(R"$\phi$")
ax.set_ylabel(R"$\cos\theta$")
ax.set_yticks([-1, 0, +1])
ax.set_yticklabels(["-1", "0", "+1"])
ax.set_xticks([0, np.pi / 2, np.pi, 3 * np.pi / 2, 2 * np.pi])
ax.set_xticklabels(
    ["0", R"$\frac{1}{2}\pi$", R"$\pi$", R"$\frac{3}{2}\pi$", R"$2\pi$"]
)
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False

MESH = None


def plot(*, Phi_val, Py_val, **mag_phase):
    global MESH
    amplitudes = [
        mag * np.exp(mag_phase[f"{k[:-4]}_phi"] * 1j)
        for k, mag in mag_phase.items()
        if k.endswith("mag")
    ]
    Z = intensity_func(
        theta_array,
        phi_array,
        Phi_val,
        Py_val,
        *amplitudes,
    )

    if MESH is None:
        MESH = ax.pcolormesh(phi_array, cos_theta_array, Z, cmap=plt.cm.Reds)
        c_bar = fig.colorbar(MESH, ax=ax, pad=0.015)
        c_bar.ax.get_yaxis().labelpad = 15
        c_bar.ax.set_ylabel("Intensity (A.U.)", rotation=270)
    else:
        MESH.set_array(Z)
    Z_max = np.nanmax(Z)
    if Z_max > 0.1:
        MESH.set_clim(vmin=0, vmax=Z_max)
    fig.canvas.draw()


fig.tight_layout()
output = w.interactive_output(plot, controls=sliders)
w.VBox([UI, output])

In [ ]:
intensity_expr